In [1]:
from pprint import pprint
import warnings

from penaltyblog.matchflow import Flow, where_equals, get_index
from statsbombpy import NoAuthWarning

# Suppress NoAuthWarning warnings
warnings.filterwarnings("ignore", category=NoAuthWarning)

ImportError: cannot import name 'NoAuthWarning' from 'statsbombpy' (/Users/martin/repos/penaltyblog/venv/lib/python3.13/site-packages/statsbombpy/__init__.py)

In [ ]:
events = sb.events(8657, fmt="dict").values()

In [ ]:
type(events)

## Competitions

Get the first competition listed for Italy.

In [ ]:
result = (
    Flow.statsbomb.competitions()
    .filter(where_equals("country_name", "Italy"))
    .collect()
)

pprint(result[0])

## Matches

Get the matches for a given competition and season and filter to specific nested fields using "dot" notation.

In [ ]:
result = (
    Flow.statsbomb.matches(competition_id=1238, season_id=108)
    .select(
        "competition.competition_name", 
        "home_team.home_team_name", 
        "away_team.away_team_name", 
        "referee.name",
        "referee.country.name"
    )
    .rename(
        **{
            "competition.competition_name": "competition_name",
            "home_team.home_team_name": "home_team_name",
            "away_team.away_team_name": "away_team_name",
            "referee.name": "referee_name",
            "referee.country.name": "referee_country_name",
        }
    )
    .collect()
)

pprint(result[:3])

## Lineups

Get the first player in the lineup for each team for a given match_id.

In [ ]:
result = (
    Flow.statsbomb.lineups(match_id=8657)
    .assign(player=get_index("lineup", 0))
    .select("team_name", "player")
    .collect()
)

pprint(result)

## Events

Get all events for a given match_id and count who took the most shots.

In [ ]:
result = (
    Flow.statsbomb.events(match_id=8657)
    .filter(where_equals("type.name", "Shot"))
    .select("player.name")
    .group_by("player.name")
    .summary(n_shots="count")
    .sort("n_shots", reverse=True)
    .limit(3)
)

for record in result:
    print(record)

In [ ]:
import warnings
from statsbombpy import NoAuthWarning

# Suppress NoAuthWarning warnings
warnings.filterwarnings("ignore", category=NoAuthWarning)
